In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import quantstats as qs
import matplotlib.pyplot as plt
from module import *
from finlab import data
from finlab.ml import feature

In [50]:
def load_fundamental_data(path = './data/ifrs.csv'):
    '''
    This function loads seasonal fundamental data from CMoney.
    '''
    data = pd.read_csv(path, parse_dates=True, encoding='cp950')
    data.columns = data.columns.str.strip()
    data['年月'] = pd.to_datetime(data['年月'], format = '%Y%m').dt.to_period('M')
    data['財報發布日'] = pd.to_datetime(data['財報發布日'])
    #data[['股票代碼', '股票名稱']] = data['證券代碼'].str.extract(r'(\d+)(\D+)')
    data.set_index(['證券代碼', '財報發布日'], inplace=True)
    data.sort_index(level=0, inplace=True)
    data.drop(columns=['合併(Y/N)', '單季(Q)/單半年(H)', '月份', '季別', '幣別', '市場別', '財報附註TEJ是否完成Y/N', 
                       '財報類別（1個別2個體3合併）', '財報年月起日', '財報年月迄日', '市場別', '交易所主產業代碼',
                       '交易所子產業代碼', 'TEJ主產業代碼', 'TEJ子產業代碼', '財報附註TEJ是否完成Y/N',], inplace = True)
    data.dropna(axis=1, how='all', inplace= True)
    #data = data.reorder_levels(['股票代碼', '年月'])
    return data

In [51]:
data = load_fundamental_data()
data2 = load_fundamental_data(path = './data/ifrs_oos.csv')

In [53]:
def combine_price(*price_dfs):
    new_price = price_dfs[0]
    for i in range(1, len(price_dfs)):
        new_price = pd.concat((new_price, price_dfs[i]), axis=0)
    new_price.drop_duplicates(inplace=True)
    new_price.sort_index(level='證券代碼', inplace=True)
    return new_price

combine_price(data, data2)

年月   現金及約當現金  透過損益按公允價值衡量之金融資產－流動  \
證券代碼     財報發布日                                                
1101 台泥  2015-05-15  2015-03  31406216               146388   
         2015-08-14  2015-06  35848634               145919   
         2015-11-13  2015-09  35931378               148581   
         2016-03-31  2015-12  38977360               147113   
         2016-05-13  2016-03  31747040               146462   
...                      ...       ...                  ...   
9958 世紀鋼 2023-05-15  2023-03   3339235                   23   
         2023-08-11  2023-06   2278106                    0   
         2023-11-13  2023-09   1608540                  915   
         2024-03-08  2023-12   2000598                    0   
         2024-05-14  2024-03   1887925                 1405   

                     透過其他綜合損益按公允價值衡量之金融資產－流動  按攤銷後成本衡量之金融資產－流動  避險之金融資產－流動  \
證券代碼     財報發布日                                                               
1101 台泥  2015-05-15               19582139.0                 0         0.0   
         2015-08-14               18260651.0                 0         0.0   
         2015-11-13               16358225.0                 0         0.0   
         2016-03-31               15227906.0                 0         0.0   
         2016-05-13               14972453.0                 0         0.0   
...                                      ...               ...         ...   
9958 世紀鋼 2023-05-15                      0.0           2045590         0.0   
         2023-08-11                      0.0           1934955         0.0   
         2023-11-13                      0.0           2284558         0.0   
         2024-03-08                      0.0           2257300         0.0   
         2024-05-14                      0.0           1199052         0.0   

                       合約資產－流動     應收帳款及票據    其他應收款  資金貸予他人－流動  ...  \
證券代碼     財報發布日                                                  ...   
1101 台泥  2015-05-15        0.0  21758400.0  1900085  1260243.0  ...   
         2015-08-14        0.0  21357337.0  2288234   449311.0  ...   
         2015-11-13        0.0  19703043.0   799201   571172.0  ...   
         2016-03-31        0.0  20212952.0   781245   551198.0  ...   
         2016-05-13        0.0  15126668.0   623785   548660.0  ...   
...                        ...         ...      ...        ...  ...   
9958 世紀鋼 2023-05-15  5358686.0    693580.0   121941        0.0  ...   
         2023-08-11  5040202.0   1124859.0   395908        0.0  ...   
         2023-11-13  5459330.0   1295775.0    98760        0.0  ...   
         2024-03-08  6373009.0   1531038.0    72300        0.0  ...   
         2024-05-14  7105701.0   1084713.0    41503        0.0  ...   

                      自由現金流量(D)  季底應收帳款/營收TSE  季底存貨/營收TSE    每人營收  每人營業利益  \
證券代碼     財報發布日                                                              
1101 台泥  2015-05-15   7042711.0        101.48       46.53  2192.0   173.0   
         2015-08-14  10555579.0         85.45       39.19  2558.0   298.0   
         2015-11-13   -969249.0         86.86       40.77  2322.0   204.0   
         2016-03-31    452995.0         82.30       36.40  2500.0   312.0   
         2016-05-13   9271951.0         85.35       47.87  1804.0    49.0   
...                         ...           ...         ...     ...     ...   
9958 世紀鋼 2023-05-15  -2410616.0         25.93       47.17     NaN     NaN   
         2023-08-11   -551116.0         34.95       51.26     NaN     NaN   
         2023-11-13   -532781.0         35.55       49.07     NaN     NaN   
         2024-03-08  -1304969.0         30.77       28.01     NaN     NaN   
         2024-05-14   1055946.0         34.45       39.98     NaN     NaN   

                       每人配備率      季底普通股市值  當季季底P/E  當季季底P/B  當季季底PSR  
證券代碼     財報發布日                                                        
1101 台泥  2015-05-15  11317.0  162824918.0     8.53     1.38     1.44  
         2015-08-14  11886.0  143810216.0     8.96  